In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tweepy
import json
from config import (consumer_key, consumer_secret, 
                    access_token, access_token_secret)
from datetime import datetime, timedelta, timezone
import time

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [2]:
ffb_twitters = ['@Rotoworld_FB', '@AdamSchefter', '@sn_fantasy', '@FantasyADHD', '@PFF_Fantasy', '@FantasyPros', 
                '@MatthewBerryTMR', '@Stephania_ESPN', '@FieldYates', '@karabelleric', '@SultanofStat', 
                '@MattWaldman', '@ProFootballDoc', '@espn', '@darrenrovell', '@MattFtheOracle', '@The_Oddsmaker', 
                '@ChrisRaybon', '@FantasyLabsNFL', '@AdamLevitan', '@CSURAM88', '@BalesFootball', '@EvanSilva',
                '@LordReebs', '@GrahamBarfield', '@MattHarmon_BYB', '@CFrelund', '@ThePowerRank', '@Cole_Kev', 
                '@FriscoJosh', '@Fantasy_Mansion', '@BrandonHerFFB', '@FBallGameplan', '@ScottBarrettDFB', '@RyanMc23',
                '@MikeTagliereNFL', '@LateRoundQB', '@CDCarter13', '@LizLoza_FF', '@HarrisFootball']

In [3]:
#for loop to collect all tweet data from each fantasy twitter handle
merged_tweets = []
tweet_count = []

for x in ffb_twitters:
    num_tweets = 0
    for status in tweepy.Cursor(api.user_timeline, id=x).items(300):
        merged_tweets.append(status)
        num_tweets += 1
        tweet_count.append(num_tweets)

In [4]:
#organize tweets into a data frame
df = pd.DataFrame([x._json for x in merged_tweets])[['text', 'created_at', 'user']]
df['label'] = df.user.map(lambda x: x.get('name'))
df['tweet_count'] = tweet_count
df.head()

,text,created_at,user,label,tweet_count
0,Bills surprisingly part ways with WR Kerley ht...,Sat Sep 15 16:09:45 +0000 2018,"{'id': 20950232, 'id_str': '20950232', 'name':...",Rotoworld Football,1
1,#NFL Week 2 Injury Report: Checking in on Leon...,Sat Sep 15 15:22:05 +0000 2018,"{'id': 20950232, 'id_str': '20950232', 'name':...",Rotoworld Football,2
2,#NFL Narratives: @JessePantuosco highlights Ma...,Sat Sep 15 14:19:05 +0000 2018,"{'id': 20950232, 'id_str': '20950232', 'name':...",Rotoworld Football,3
3,"Despite his lack of production in Week 1, Trav...",Sat Sep 15 12:00:24 +0000 2018,"{'id': 20950232, 'id_str': '20950232', 'name':...",Rotoworld Football,4
4,RT @RotoPat: Lots of injury updates to the ran...,Fri Sep 14 22:06:40 +0000 2018,"{'id': 20950232, 'id_str': '20950232', 'name':...",Rotoworld Football,5


In [5]:
#double checking total tweet count to make sure we have all the data
df.describe()
# df.to_pickle(df)

,tweet_count
count,12000.000000
mean,150.500000
std,86.605668
min,1.000000
25%,75.750000
50%,150.500000
75%,225.250000
max,300.000000


In [6]:
analyzer = SentimentIntensityAnalyzer() # Initialize the class

In [7]:
merged_text = [x._json['text'] for x in merged_tweets]

In [8]:
#running vader analysis for all tweets
compound_list = []
positive_list = []
negative_list = []
neutral_list = []

for tweet in merged_text:
    vs = analyzer.polarity_scores(tweet)
    compound_list.append(vs.get('compound'))
    positive_list.append(vs.get('pos'))
    negative_list.append(vs.get('neg'))
    neutral_list.append(vs.get('neu'))

In [9]:
#adding vader scores to the df
df['compound_score'] = compound_list
df['positive_score'] = positive_list
df['negative_score'] = negative_list
df['neutral_score'] = neutral_list
df.head()

,text,created_at,user,label,tweet_count,compound_score,positive_score,negative_score,neutral_score
0,Bills surprisingly part ways with WR Kerley ht...,Sat Sep 15 16:09:45 +0000 2018,"{'id': 20950232, 'id_str': '20950232', 'name':...",Rotoworld Football,1,0.2960,0.239,0.000,0.761
1,#NFL Week 2 Injury Report: Checking in on Leon...,Sat Sep 15 15:22:05 +0000 2018,"{'id': 20950232, 'id_str': '20950232', 'name':...",Rotoworld Football,2,-0.4215,0.000,0.157,0.843
2,#NFL Narratives: @JessePantuosco highlights Ma...,Sat Sep 15 14:19:05 +0000 2018,"{'id': 20950232, 'id_str': '20950232', 'name':...",Rotoworld Football,3,-0.7506,0.000,0.348,0.652
3,"Despite his lack of production in Week 1, Trav...",Sat Sep 15 12:00:24 +0000 2018,"{'id': 20950232, 'id_str': '20950232', 'name':...",Rotoworld Football,4,-0.0258,0.086,0.090,0.824
4,RT @RotoPat: Lots of injury updates to the ran...,Fri Sep 14 22:06:40 +0000 2018,"{'id': 20950232, 'id_str': '20950232', 'name':...",Rotoworld Football,5,-0.0258,0.110,0.114,0.776


In [10]:
elliot_df = df[df['text'].str.contains("Elliot")]
elliot_df.head()

,text,created_at,user,label,tweet_count,compound_score,positive_score,negative_score,neutral_score
2539,@Elliot_Solop From @vineyardvines !,Sun Sep 09 15:52:16 +0000 2018,"{'id': 58919137, 'id_str': '58919137', 'name':...",Field Yates,140,0.0000,0.000,0.000,1.000
4432,@BudElliott3 😂😂😂,Fri Sep 07 17:53:23 +0000 2018,"{'id': 24277551, 'id_str': '24277551', 'name':...",Darren Rovell,233,0.0000,0.000,0.000,1.000
4629,RT @DraftKings: WATCH: How are Ezekiel Elliott...,Fri Aug 24 16:58:48 +0000 2018,"{'id': 1265991506, 'id_str': '1265991506', 'na...",Matthew Freedman,130,-0.5574,0.000,0.204,0.796
7224,Ezekiel Elliott’s continued his encouraging pa...,Mon Sep 10 14:40:57 +0000 2018,"{'id': 1852555963, 'id_str': '1852555963', 'na...",Graham Barfield,25,0.7003,0.279,0.000,0.721
7434,Ezekiel Elliott’s routes run per game spiked l...,Wed Aug 01 23:05:35 +0000 2018,"{'id': 1852555963, 'id_str': '1852555963', 'na...",Graham Barfield,235,0.0000,0.000,0.000,1.000


In [11]:
elliot_df['text'].tolist()

['@Elliot_Solop From @vineyardvines !',
 '@BudElliott3 😂😂😂',
 "RT @DraftKings: WATCH: How are Ezekiel Elliott &amp; Dak Prescott affected by injuries to the Cowboys' offensive line? \n\n@dklive's @reidtfowler…",
 'Ezekiel Elliott’s continued his encouraging pass-down involvement in Week 1 for #Cowboys. Elliott played on 92% of… https://t.co/LcxwyNKlrR',
 'Ezekiel Elliott’s routes run per game spiked last year (26.7) compared to his rookie season (17.6). In fact, Zeke r… https://t.co/A7rPKhhrS2',
 'RT @BudElliott3: Abdul Bello.\n\nFSU is now without Landon Dickerson, Cole Minshew, and Derrick Kelly and they are in big trouble because tac…',
 'RT @SBRSportsPicks: Saturday Morning at the #IFBC2018: @SBN_BillC, @BudElliott3 &amp; @thepowerrank sat down with @DrewMartinBets to preview #C…',
 'RT @SBRSportsPicks: Getting ready to go LIVE! \n@thepowerrank @SBN_BillC @BudElliott3 &amp; @DrewMartinBets are waiting to present at #IFBC2018…',
 'Over/Under 19.5 Adrian Peterson Rush Attempts?\nOver

In [12]:
#getting average sentiment score for a player
elliot_df.get('compound_score').mean()

0.017877777777777784

In [13]:
matt_df = df[df['text'].str.contains('Stafford')]
matt_df.head()

,text,created_at,user,label,tweet_count,compound_score,positive_score,negative_score,neutral_score
215,Stafford melts down with 4 INTs against Jets h...,Tue Sep 11 02:15:12 +0000 2018,"{'id': 20950232, 'id_str': '20950232', 'name':...",Rotoworld Football,216,0.0000,0.000,0.000,1.000
1599,Even Matthew Stafford thinks Derek Carr just t...,Tue Sep 11 05:07:29 +0000 2018,"{'id': 185125451, 'id_str': '185125451', 'name...",FantasyPros,100,-0.5423,0.000,0.259,0.741
1615,Matthew Stafford has robbed us all of #Jets jo...,Tue Sep 11 01:22:00 +0000 2018,"{'id': 185125451, 'id_str': '185125451', 'name...",FantasyPros,116,0.2500,0.154,0.000,0.846
1894,Stafford was off but Jets D is legit. https://...,Tue Sep 11 02:23:13 +0000 2018,"{'id': 20899023, 'id_str': '20899023', 'name':...",Matthew Berry,95,0.0000,0.000,0.000,1.000
1897,For those of you with Stafford at QB... https:...,Tue Sep 11 01:37:50 +0000 2018,"{'id': 20899023, 'id_str': '20899023', 'name':...",Matthew Berry,98,0.0000,0.000,0.000,1.000


In [14]:
matt_df.get('compound_score').mean()

0.10738965517241379

In [15]:
bell_df = df[df['text'].str.contains("Le'Veon Bell")]
bell_df

,text,created_at,user,label,tweet_count,compound_score,positive_score,negative_score,neutral_score
38,"""Gotta be honest, I think Le'Veon Bell is goin...",Fri Sep 14 17:14:05 +0000 2018,"{'id': 20950232, 'id_str': '20950232', 'name':...",Rotoworld Football,39,0.5106,0.148,0.000,0.852
166,Le'Veon Bell not with Steelers on Wednesday ht...,Wed Sep 12 13:51:36 +0000 2018,"{'id': 20950232, 'id_str': '20950232', 'name':...",Rotoworld Football,167,0.0000,0.000,0.000,1.000
300,"It's just after noon, and there's no sign of R...",Sat Sep 15 16:08:20 +0000 2018,"{'id': 51263592, 'id_str': '51263592', 'name':...",Adam Schefter,1,-0.2960,0.000,0.095,0.905
421,To anyone suggesting the Steelers trade RB Le'...,Sun Sep 09 14:07:12 +0000 2018,"{'id': 51263592, 'id_str': '51263592', 'name':...",Adam Schefter,122,0.0000,0.000,0.000,1.000
435,The 4 p.m. deadline today will come and go wit...,Sat Sep 08 19:23:24 +0000 2018,"{'id': 51263592, 'id_str': '51263592', 'name':...",Adam Schefter,136,0.0000,0.000,0.000,1.000
469,RT @FieldYates: Le'Veon Bell's agent Adisa Bak...,Wed Sep 05 18:18:50 +0000 2018,"{'id': 51263592, 'id_str': '51263592', 'name':...",Adam Schefter,170,0.6124,0.185,0.000,0.815
470,The latest on Le'Veon Bell. https://t.co/Timql...,Wed Sep 05 16:46:54 +0000 2018,"{'id': 51263592, 'id_str': '51263592', 'name':...",Adam Schefter,171,0.0000,0.000,0.000,1.000
476,RT @RossTuckerNFL: From a strictly business pe...,Wed Sep 05 11:22:04 +0000 2018,"{'id': 51263592, 'id_str': '51263592', 'name':...",Adam Schefter,177,0.2144,0.078,0.000,0.922
505,"No Le’Veon Bell, still.\n\nSteelers GM Kevin C...",Mon Sep 03 15:24:39 +0000 2018,"{'id': 51263592, 'id_str': '51263592', 'name':...",Adam Schefter,206,-0.6486,0.000,0.227,0.773
652,"Fantasy Football Updates: Greg Olsen, Le'Veon ...",Sun Sep 09 06:33:05 +0000 2018,"{'id': 30687192, 'id_str': '30687192', 'name':...",Sporting News Fantasy,53,0.0000,0.000,0.000,1.000


In [16]:
bell_df.get('compound_score').mean()

0.045875000000000006

In [17]:
#creating a string of just the created_at info I want
str(df['created_at'].iloc[0][4:10]+' '+df['created_at'].iloc[0][26:30])

'Sep 15 2018'

In [18]:
#convert to datetime
datetime.strptime('Sep 12 2018', '%b %d %Y')

datetime.datetime(2018, 9, 12, 0, 0)

In [19]:
#creating new column of just date information
df['date'] = df['created_at'].map(lambda x: str(x[4:10]+' '+x[26:30]))

In [20]:
#creating date column to datetime in new column
df['datetime'] = df['date'].map(lambda x: datetime.strptime(x, '%b %d %Y'))

In [21]:
df['datetime'].iloc[0]

Timestamp('2018-09-15 00:00:00')

In [ ]:
#can I do this dynamically?????????????????????????????????????
one_week_ago = datetime.strptime('Sep 8 2018', '%b %d %Y')

In [23]:
#creating df of only tweets within one week
recent_tweets = df[df['datetime']>one_week_ago]

In [24]:
#checking total number of tweets
recent_tweets.describe()

,tweet_count,compound_score,positive_score,negative_score,neutral_score
count,7847.000000,7847.000000,7847.000000,7847.000000,7847.000000
mean,123.374920,0.099743,0.101031,0.048228,0.850486
std,83.528796,0.339200,0.156255,0.102322,0.175701
min,1.000000,-0.946300,0.000000,0.000000,0.000000
25%,51.000000,0.000000,0.000000,0.000000,0.748000
50%,111.000000,0.000000,0.000000,0.000000,0.888000
75%,188.000000,0.361200,0.170000,0.000000,1.000000
max,300.000000,0.953800,1.000000,0.812000,1.000000


In [25]:
recent_tweets.head()

,text,created_at,user,label,tweet_count,compound_score,positive_score,negative_score,neutral_score,date,datetime
0,Bills surprisingly part ways with WR Kerley ht...,Sat Sep 15 16:09:45 +0000 2018,"{'id': 20950232, 'id_str': '20950232', 'name':...",Rotoworld Football,1,0.2960,0.239,0.000,0.761,Sep 15 2018,2018-09-15
1,#NFL Week 2 Injury Report: Checking in on Leon...,Sat Sep 15 15:22:05 +0000 2018,"{'id': 20950232, 'id_str': '20950232', 'name':...",Rotoworld Football,2,-0.4215,0.000,0.157,0.843,Sep 15 2018,2018-09-15
2,#NFL Narratives: @JessePantuosco highlights Ma...,Sat Sep 15 14:19:05 +0000 2018,"{'id': 20950232, 'id_str': '20950232', 'name':...",Rotoworld Football,3,-0.7506,0.000,0.348,0.652,Sep 15 2018,2018-09-15
3,"Despite his lack of production in Week 1, Trav...",Sat Sep 15 12:00:24 +0000 2018,"{'id': 20950232, 'id_str': '20950232', 'name':...",Rotoworld Football,4,-0.0258,0.086,0.090,0.824,Sep 15 2018,2018-09-15
4,RT @RotoPat: Lots of injury updates to the ran...,Fri Sep 14 22:06:40 +0000 2018,"{'id': 20950232, 'id_str': '20950232', 'name':...",Rotoworld Football,5,-0.0258,0.110,0.114,0.776,Sep 14 2018,2018-09-14


In [26]:
#checking to make sure that the date filter worked
recent_tweets['datetime'].value_counts()

2018-09-14    1298
2018-09-13    1155
2018-09-10    1090
2018-09-11    1060
2018-09-12    1019
2018-09-09    1007
2018-09-07     516
2018-09-15     356
2018-09-08     346
Name: datetime, dtype: int64

In [27]:
recent_tweets = recent_tweets.reset_index()

In [28]:
recent_tweets['text'].iloc[0]

'Bills surprisingly part ways with WR Kerley https://t.co/wTlWFqNNAq'

In [29]:
bell_df = [recent_tweets['text'].str.contains("Le'Veon Bell")]

In [30]:
bell_df.head()

AttributeError: 'list' object has no attribute 'head'

In [31]:
player_df = pd.read_csv('player_list.csv', encoding ='latin1')

In [32]:
player_df.head()

,number,player
0,1,Cam Newton
1,2,Drew Brees
2,3,Aaron Rodgers
3,4,Alex Smith
4,5,Ben Roethlisberger


In [33]:
player_df['player'][0][1:]

'Cam Newton'

In [34]:
player_df['player'] = player_df['player'].map(lambda x: x[1:])

In [35]:
player_df['player'].iloc[0]

'Cam Newton'

In [36]:
#for each player in the league, need to create a mask for each to get compound score, get mean compound score and append to list
compound_scores = []
for player in player_df['player']:
    temp_df = recent_tweets[recent_tweets['text'].str.contains(player)]
    score_mean = temp_df.get('compound_score').mean()
    compound_scores.append(score_mean)

In [37]:
#input mean compound score into player table
player_df['compound_score'] = compound_scores
player_df.head()

,number,player,compound_score
0,1,Cam Newton,-0.109200
1,2,Drew Brees,0.296667
2,3,Aaron Rodgers,-0.095170
3,4,Alex Smith,0.154164
4,5,Ben Roethlisberger,-0.018416


In [38]:
player_df['compound_score'].mean()

0.06883564229574976

In [39]:
#creat bins for positive, neutral, and negative (awesome, good, neutral, concerned, terrible)
bins = [-1, -.5, -.02, .02, .5,  1]
player_df['binned'] = pd.cut(player_df['compound_score'], bins)
player_df.head()

,number,player,compound_score,binned
0,1,Cam Newton,-0.109200,"(-0.5, -0.02]"
1,2,Drew Brees,0.296667,"(0.02, 0.5]"
2,3,Aaron Rodgers,-0.095170,"(-0.5, -0.02]"
3,4,Alex Smith,0.154164,"(0.02, 0.5]"
4,5,Ben Roethlisberger,-0.018416,"(-0.02, 0.02]"


In [40]:
player_df['binned'] = player_df['binned'].map(lambda x: str(x))

In [41]:
player_df['binned'].iloc[0]

'(-0.5, -0.02]'

In [42]:
player_df['binned'].value_counts()

(0.02, 0.5]      85
(-0.5, -0.02]    41
(-0.02, 0.02]    21
(0.5, 1.0]        4
(-1.0, -0.5]      2
Name: binned, dtype: int64

In [52]:
sent_bins = {
    '(-1.0, -0.5]': 'Terrrible',
    '(-0.5, -0.02]': 'Concerned',
    '(-0.02, 0.02]': 'Neutral',
    '(0.02, 0.5]': 'Good',
    '(0.5, 1.0]': 'Awesome'
}

In [53]:
player_df['expert_sentiment'] = player_df['binned'].map(sent_bins)
player_df.head()

,number,player,compound_score,binned,expert_sentiment
0,1,Cam Newton,-0.109200,"(-0.5, -0.02]",Concerned
1,2,Drew Brees,0.296667,"(0.02, 0.5]",Good
2,3,Aaron Rodgers,-0.095170,"(-0.5, -0.02]",Concerned
3,4,Alex Smith,0.154164,"(0.02, 0.5]",Good
4,5,Ben Roethlisberger,-0.018416,"(-0.02, 0.02]",Neutral


In [54]:
player_df['expert_sentiment'].value_counts()

Good         85
Concerned    41
Neutral      21
Awesome       4
Terrrible     2
Name: expert_sentiment, dtype: int64

In [55]:
player_df.to_csv('expert_tweets.csv')

In [ ]:
#end goal = create csv with each player and their compound score. user can query for a player and get a result which shows how
# the fantasy experts feel about that player (positive, negative, or neutral)

In [ ]:
#want to sort by just the last week's data to get current tweets
# one_week_ago = datetime.today() - timedelta(days=7)
# one_week_ago